Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
## Logistic classifier with L2 regularization

batch_size = 1000
#batch_size = 128 # this overfits the data
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Weights and biases
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Regularization
  regularizers = tf.nn.l2_loss(logits - tf_train_labels)
  regularizers = tf.nn.l2_loss(weights)
  loss += beta *regularizers  
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)    

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.233561
Minibatch accuracy: 12.9%
Validation accuracy: 15.6%
Minibatch loss at step 500: 2.516944
Minibatch accuracy: 79.4%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 0.999761
Minibatch accuracy: 81.9%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 0.740815
Minibatch accuracy: 83.1%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 0.711244
Minibatch accuracy: 82.6%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.695592
Minibatch accuracy: 81.7%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 0.635043
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Test accuracy: 89.8%


In [7]:
## Hidden Neural Network with L2 regularization

batch_size = 128
#batch_size = 128 # this overfits the data
hidden_neurons = 1024
beta= 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of the hidden and output layers
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_neurons]))
  hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([hidden_neurons, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    # The hidden layer.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)  
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  logits = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  regularizers = tf.nn.l2_loss(logits - tf_train_labels)
  regularizers = (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights))
  loss += beta *regularizers  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf_valid_dataset, weights) + biases)    
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_weights) + hidden_biases)  
  logits_valid = tf.matmul(hidden_valid, output_weights) + output_biases
  valid_prediction = tf.nn.softmax(logits_valid)
    
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_weights) + hidden_biases)  
  logits_test = tf.matmul(hidden_test, output_weights) + output_biases
  test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1231.024658
Minibatch accuracy: 16.4%
Validation accuracy: 23.4%
Minibatch loss at step 500: 209.046783
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 46.089478
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Minibatch loss at step 1500: 10.555794
Minibatch accuracy: 90.6%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 2.777631
Minibatch accuracy: 91.4%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 1.072010
Minibatch accuracy: 88.3%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 0.746860
Minibatch accuracy: 83.6%
Validation accuracy: 86.0%
Test accuracy: 92.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
## Hidden Neural Network with L2 regularization

batch_size = 10
#batch_size = 128 # this overfits the data
hidden_neurons = 1024
beta= 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of the hidden and output layers
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_neurons]))
  hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([hidden_neurons, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    # The hidden layer.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)  
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  logits = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  regularizers = tf.nn.l2_loss(logits - tf_train_labels)
  regularizers = (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights))
  loss += beta *regularizers  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf_valid_dataset, weights) + biases)    
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_weights) + hidden_biases)  
  logits_valid = tf.matmul(hidden_valid, output_weights) + output_biases
  valid_prediction = tf.nn.softmax(logits_valid)
    
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_weights) + hidden_biases)  
  logits_test = tf.matmul(hidden_test, output_weights) + output_biases
  test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1262.387573
Minibatch accuracy: 0.0%
Validation accuracy: 17.7%
Minibatch loss at step 500: 32223.281250
Minibatch accuracy: 30.0%
Validation accuracy: 52.7%
Minibatch loss at step 1000: 9963.121094
Minibatch accuracy: 70.0%
Validation accuracy: 58.1%
Minibatch loss at step 1500: 2888.139160
Minibatch accuracy: 60.0%
Validation accuracy: 40.5%
Minibatch loss at step 2000: 629.920227
Minibatch accuracy: 10.0%
Validation accuracy: 28.5%
Minibatch loss at step 2500: 138.992996
Minibatch accuracy: 40.0%
Validation accuracy: 27.6%
Minibatch loss at step 3000: 32.914162
Minibatch accuracy: 10.0%
Validation accuracy: 27.5%
Test accuracy: 29.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [15]:
## Hidden Neural Network with L2 regularization

batch_size = 128
#batch_size = 128 # this overfits the data
hidden_neurons = 1024
beta= 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of the hidden and output layers
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_neurons]))
  hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([hidden_neurons, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    # The hidden layer.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)  
  hidden = tf.nn.dropout(hidden,0.5)    
  logits = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  regularizers = tf.nn.l2_loss(logits - tf_train_labels)
  regularizers = (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights))
  loss += beta *regularizers  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf_valid_dataset, weights) + biases)    
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_weights) + hidden_biases)  
  logits_valid = tf.matmul(hidden_valid, output_weights) + output_biases
  valid_prediction = tf.nn.softmax(logits_valid)
    
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_weights) + hidden_biases)  
  logits_test = tf.matmul(hidden_test, output_weights) + output_biases
  test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1409.233398
Minibatch accuracy: 8.6%
Validation accuracy: 30.5%
Minibatch loss at step 500: 214.150757
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 46.488770
Minibatch accuracy: 80.5%
Validation accuracy: 83.6%
Minibatch loss at step 1500: 10.665809
Minibatch accuracy: 89.1%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 2.860831
Minibatch accuracy: 90.6%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 1.165211
Minibatch accuracy: 85.2%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 0.881053
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Test accuracy: 91.7%


In [16]:
## Hidden Neural Network with L2 regularization

batch_size = 128
#batch_size = 128 # this overfits the data
hidden_neurons = 1024
beta= 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of the hidden and output layers
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_neurons]))
  hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([hidden_neurons, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    # The hidden layer.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)  
  hidden = tf.nn.dropout(hidden,0.5)    
  logits = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  regularizers = tf.nn.l2_loss(logits - tf_train_labels)
  regularizers = (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights))
  loss += beta *regularizers  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf_valid_dataset, weights) + biases)    
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_weights) + hidden_biases)  
  logits_valid = tf.matmul(hidden_valid, output_weights) + output_biases
  valid_prediction = tf.nn.softmax(logits_valid)
    
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_weights) + hidden_biases)  
  logits_test = tf.matmul(hidden_test, output_weights) + output_biases
  test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1381.156860
Minibatch accuracy: 10.2%
Validation accuracy: 32.0%
Minibatch loss at step 500: 213.307159
Minibatch accuracy: 78.9%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 46.333332
Minibatch accuracy: 78.9%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 10.611371
Minibatch accuracy: 89.1%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 2.853436
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 1.187856
Minibatch accuracy: 83.6%
Validation accuracy: 85.0%
Minibatch loss at step 3000: 0.848677
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 3500: 0.752115
Minibatch accuracy: 82.8%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 0.603125
Minibatch accuracy: 87.5%
Validation accuracy: 85.2%
Minibatch loss at step 4500: 0.638214
Minibatch accuracy: 84.4%
Validation accuracy: 85.1%
Minibatch loss at step 5000: 0.653146
Minibatch accuracy: 85.9%
Validation 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
## Hidden Neural Network with L2 regularization

batch_size = 128
#batch_size = 128 # this overfits the data
hidden_neurons = 1024
beta= 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of the hidden and output layers
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_neurons]))
  hidden_biases = tf.Variable(tf.zeros([hidden_neurons]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([hidden_neurons, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    # The hidden layer.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)  
  hidden = tf.nn.dropout(hidden,0.5)    
  logits = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  regularizers = tf.nn.l2_loss(logits - tf_train_labels)
  regularizers = (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights))
  loss += beta *regularizers  

  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Optimizer with learning decay
  global_step = tf.Variable(0)  # count the number of steps taken.  
  learning_rate = tf.train.exponential_decay(0.5, step, 300, 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf_valid_dataset, weights) + biases)    
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_weights) + hidden_biases)  
  logits_valid = tf.matmul(hidden_valid, output_weights) + output_biases
  valid_prediction = tf.nn.softmax(logits_valid)
    
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_weights) + hidden_biases)  
  logits_test = tf.matmul(hidden_test, output_weights) + output_biases
  test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1410.007080
Minibatch accuracy: 10.9%
Validation accuracy: 26.6%
Minibatch loss at step 100: 828.673218
Minibatch accuracy: 71.9%
Validation accuracy: 80.1%
Minibatch loss at step 200: 710.110718
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 300: 564.885010
Minibatch accuracy: 71.9%
Validation accuracy: 81.2%